In [ ]:
!pip install skfeature-chappers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from collections import Counter
from sklearn.utils import resample
from skfeature.function.similarity_based import fisher_score
import numpy as np
import pandas as pd
import random
import math,time,sys, os
from matplotlib import pyplot
from datetime import datetime

!pip install ReliefF

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 KB 2.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 KB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ReliefF: filename=ReliefF-0.1.2-py3-none-any.whl size=6318 sha256=bad69699655e5bb99631777e99d13367b6086b942db11c5c66de459acf0166be
  Stored in directory: /root/.cache/pip/wheels/d8/90/ef/7691706c1c418d2c266a0b3b06371900f4b86e74223912c563
Successfully built ReliefF


In [ ]:
!pip install --ignore-installed scikit-learn
!pip install --upgrade numpy
!pip install --ignore-installed skfeature-chappers
# !pip install --upgrade numpy
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from collections import Counter
from sklearn.utils import resample
from skfeature.function.similarity_based import fisher_score
import numpy as np
import pandas as pd
import random
import math,time,sys, os
from matplotlib import pyplot
from datetime import datetime

!pip install tabulate
!pip install --ignore-installed ReliefF

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 KB 23.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached skfeature_chappers-1.1.0-py3-none-any.whl (66 kB)
  Using cached numpy-1.24.2-cp38-cp38-manylinux_2_1

In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier as KNN
import matplotlib.pyplot as plt

class Solution():    
    #structure of the solution 
    def __init__(self):
        self.num_features = None
        self.num_agents = None
        self.max_iter = None
        self.obj_function = None
        self.execution_time = None
        self.convergence_curve = {}
        self.best_agent = None
        self.best_fitness = None
        self.best_accuracy = None
        self.final_population = None
        self.final_fitness = None
        self.final_accuracy = None


class Data():
    # structure of the training data
    def __init__(self):
        self.train_X = None
        self.train_Y = None
        self.val_X = None
        self.val_Y = None



def initialize(num_agents, num_features):
    # define min and max number of features
    min_features = int(0.3 * num_features)
    max_features = int(0.6 * num_features)

    # initialize the agents with zeros
    agents = np.zeros((num_agents, num_features))

    # select random features for each agent
    for agent_no in range(num_agents):

        # find random indices
        cur_count = np.random.randint(min_features, max_features)
        temp_vec = np.random.rand(1, num_features)
        temp_idx = np.argsort(temp_vec)[0][0:cur_count]

        # select the features with the ranom indices
        agents[agent_no][temp_idx] = 1   

    return agents



def sort_agents(agents, obj, data, fitness=None):
    # sort the agents according to fitness
    train_X, val_X, train_Y, val_Y = data.train_X, data.val_X, data.train_Y, data.val_Y
    (obj_function, weight_acc) = obj
   
    if fitness is None:
        # if there is only one agent
        if len(agents.shape) == 1:
            num_agents = 1
            fitness = obj_function(agents, train_X, val_X, train_Y, val_Y, weight_acc)
            return agents, fitness

        # for multiple agents
        else:
            num_agents = agents.shape[0]
            fitness = np.zeros(num_agents)
            for id, agent in enumerate(agents):
                fitness[id] = obj_function(agent, train_X, val_X, train_Y, val_Y, weight_acc)

    idx = np.argsort(-fitness)
    sorted_agents = agents[idx].copy()
    sorted_fitness = fitness[idx].copy()

    return sorted_agents, sorted_fitness



def display(agents, fitness, agent_name='Agent'):
    # display the population
    print('\nNumber of agents: {}'.format(agents.shape[0]))
    print('\n------------- Best Agent ---------------')
    print('Fitness: {}'.format(fitness[0]))
    print('Number of Features: {}'.format(int(np.sum(agents[0]))))
    print('----------------------------------------\n')

    for id, agent in enumerate(agents):
        print('{} {} - Fitness: {}, Number of Features: {}'.format(agent_name, id+1, fitness[id], int(np.sum(agent))))

    print('================================================================================\n')



def compute_accuracy(agent, train_X, test_X, train_Y, test_Y): 
    # compute classification accuracy of the given agents
    cols = np.flatnonzero(agent)     
    if(cols.shape[0] == 0):
        return 0    
    clf = KNN()

    train_data = train_X[:,cols]
    train_label = train_Y
    test_data = test_X[:,cols]
    test_label = test_Y

    clf.fit(train_data,train_label)
    acc = clf.score(test_data,test_label)

    return acc
        

def compute_fitness(agent, train_X, test_X, train_Y, test_Y, weight_acc=0.9):
    # compute a basic fitness measure
    if(weight_acc == None):
        weight_acc = 0.9

    weight_feat = 1 - weight_acc
    num_features = agent.shape[0]
    
    acc = compute_accuracy(agent, train_X, test_X, train_Y, test_Y)
    feat = (num_features - np.sum(agent))/num_features

    fitness = weight_acc * acc + weight_feat * feat
    return fitness


def Conv_plot(convergence_curve):
    # plot convergence curves
    num_iter = len(convergence_curve['fitness'])
    iters = np.arange(num_iter) + 1
    fig, axes = plt.subplots(1)
    fig.tight_layout(pad = 5) 
    fig.suptitle('Convergence Curves')
    
    axes.set_title('Convergence of Fitness over Iterations')
    axes.set_xlabel('Iteration')
    axes.set_ylabel('Avg. Fitness')
    axes.plot(iters, convergence_curve['fitness'])

    return fig, axes

In [ ]:
import numpy as np

def sigmoid(val):
    if val < 0:
        return 1 - 1/(1 + np.exp(val))
    else:
        return 1/(1 + np.exp(-val))


def v_func(val):
    return abs(val/(np.sqrt(1 + val*val)))


def u_func(val):
    alpha, beta = 2, 1.5
    return abs(alpha * np.power(abs(val), beta))


def get_trans_function(shape):
    if (shape.lower() == 's'):
        return sigmoid

    elif (shape.lower() == 'v'):
        return v_func

    elif(shape.lower() == 'u'):
        return u_func

    else:
        print('\n[Error!] We don\'t currently support {}-shaped transfer functions...\n'.format(shape))
        exit(1)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from collections import Counter
from sklearn.utils import resample
from skfeature.function.similarity_based import fisher_score
import numpy as np
import pandas as pd
import random
import math,time,sys, os
from matplotlib import pyplot
from datetime import datetime
!pip install ReliefF
predicted=[]

def chaos_map(num_features):
    firs=np.random.random()
    logis_map=np.random.rand(1,num_features)
    for i in range(num_features):
        k=4*firs*(1-firs)
        logis_map[0][i]=k
        firs=k
    return logis_map
def initialize(num_agents, num_features):
    # define min and max number of features
    min_features = int(0.3 * num_features)
    max_features = int(0.6 * num_features)

    # initialize the agents with zeros
    agents = np.zeros((num_agents, num_features))

    # select random features for each agent
    for agent_no in range(num_agents):

        # find random indices
        cur_count = np.random.randint(min_features, max_features)
        temp_vec = chaos_map(num_features)
        temp_idx = np.argsort(temp_vec)[0][0:cur_count]

        # select the features with the ranom indices
        agents[agent_no][temp_idx] = 1   

    return agents

def HS(num_agents, max_iter, train_data, train_label, obj_function = compute_fitness, save_conv_graph = False):
    def mutate(agent):
        percent=0.2
        numChange=int(num_features*percent)
        pos=np.random.randint(0,num_features-1,numChange) #choose random positions to be mutated
        agent[pos]=1-agent[pos] #mutation
        return agent
    def LAHC(particle,lamb,targ):
                _lambda = lamb #upper limit on number of iterations in LAHC
                target_fitness = compute_fitness(particle,data.train_X, data.val_X, data.train_Y, data.val_Y) #original fitness
                for i in range(_lambda):
                        new_particle = mutate(particle) #first mutation
                        temp = compute_fitness(new_particle,data.train_X, data.val_X, data.train_Y, data.val_Y)
                        if temp > target_fitness:
                            particle = new_particle.copy() #updation
                            target_fitness = temp
                        elif (temp>=(1-targ)*target_fitness):
                            temp_particle = new_particle.copy()
                            for j in range(_lambda):
                                temp_particle1 = mutate(temp_particle) #second mutation
                                temp_fitness = compute_fitness(temp_particle1,data.train_X, data.val_X, data.train_Y, data.val_Y)
                                if temp_fitness > target_fitness:
                                    target_fitness=temp_fitness
                                    particle=temp_particle1.copy() #updation
                                    break
                return particle 
    # Harmony Search Algorithm

  #  _particle1,data.train_X, data.val_X, data.train_Y, data.val_Y)
  #                               if temp_fitness < target_fitness:
  #                                   target_fitness=temp_fitness
  #                                   particle=temp_particle1.copy() #updation
  #                                   break
    #             return particle 
    # # Harmony Search Algorithm
    ############################### Parameters ####################################
    #                                                                             #
    #   num_agents: number of harmonies                                           #
    #   max_iter: maximum number of generations                                   #
    #   train_data: training samples of data                                      #
    #   train_label: class labels for the training samples                        #                
    #   obj_function: the function to maximize while doing feature selection      #
    #   trans_function_shape: shape of the transfer function used                 #
    #   save_conv_graph: boolean value for saving convergence graph               #
    #                                                                             #
    ###############################################################################
    #for a in [0.8,0.9,0.95]: b: for b in [10,15,20]:    or c in [0.1,0.2,0.3]:

    # <STEPS OF HARMOMY SEARCH ALGORITH>
    # Step 1. Initialize a Harmony Memory (HM).
    # Step 2. Improvise a new harmony from HM.
    # Step 3. If the new harmony is better than minimum harmony in HM, include the new harmony in HM, and exclude the minimum harmony from HM.
    # Step 4. If stopping criteria are not satisfied, go to Step 2.
    for a in [0.95]:
        for b in [20]:
            for c in [0.3]:
                print(f'\n Hi Guys welcome to HS+LAHC.BTW\n HMCR: {a}\n Lambda: {b}\n Targ Fit: {c}\n')
                short_name = 'HS'
                agent_name = 'Harmony'
                train_data, train_label = np.array(train_data), np.array(train_label)
                num_features = train_data.shape[1]

                # setting up the objectives
                weight_acc = None
                if(obj_function==compute_fitness):
                    # weight_acc = float(input('Weight for the classification accuracy [0-1]: '))
                    weight_acc=0.95
                obj = (obj_function, weight_acc)
                compute_accuracy = (compute_fitness, 1) # compute_accuracy is just compute_fitness with accuracy weight as 1

                # intialize the harmonies and Leader (the agent with the max fitness)
                harmonyMemory = initialize(num_agents, num_features)
                fitness = np.zeros(num_agents)
                accuracy = np.zeros(num_agents)
                Leader_agent = np.zeros((1, num_features))
                Leader_fitness = float("-inf")
                Leader_accuracy = float("-inf")
                HMCR = a     # Harmony Memory Consideration Rate

                # initialize convergence curves
                convergence_curve = {}
                convergence_curve['fitness'] = np.zeros(max_iter)

                # initialize data class
                data = Data()
                # val_size = float(input('Enter the percentage of data wanted for valdiation [0, 100]: '))/100
                val_size =0.2

                data.train_X, data.val_X, data.train_Y, data.val_Y = train_test_split(train_data, train_label, stratify=train_label, test_size=val_size)

                # create a solution object
                solution = Solution()
                solution.num_agents = num_agents
                solution.max_iter = max_iter
                solution.num_features = num_features
                solution.obj_function = obj_function

                # start timer
                start_time = time.time()

                # calculate initial fitess and sort the harmony memory and rank them
                harmonyMemory, fitness = sort_agents(harmonyMemory, obj, data)
                mean_best=0
                # create new harmonies in each iteration
                for iter_no in range(max_iter):
                    print('\n================================================================================')
                    print('                          Iteration - {}'.format(iter_no + 1))
                    print('================================================================================\n')
                    HMCR_randValue = np.random.rand()
                    newHarmony = np.zeros([1, num_features])

                    # print(HMCR)
                    # print(HMCR_randValue)

                    if HMCR_randValue <= HMCR:
                        for featureNum in range(num_features):
                            selectedAgent = random.randint(0, num_agents - 1)
                            newHarmony[0, featureNum] = harmonyMemory[selectedAgent, featureNum]

                    else:
                        for featureNum in range(num_features):
                            newHarmony[0, featureNum] = random.randint(0, 1)

                    fitnessHarmony = obj_function(newHarmony, data.train_X, data.val_X, data.train_Y, data.val_Y)

                    if fitness[num_agents-1] < fitnessHarmony:
                        harmonyMemory[num_agents-1, :] = newHarmony
                        fitness[num_agents-1] = fitnessHarmony
                    for i in range(len(harmonyMemory)):
                        harmonyMemory[i]=LAHC(harmonyMemory[i],b,c)

                    # sort harmony memory
                    harmonyMemory, fitness = sort_agents(harmonyMemory, obj, data)
                    
                    if fitness[0] > Leader_fitness:
                        Leader_agent = harmonyMemory[0].copy()
                        Leader_fitness = fitness[0].copy()
                    if np.mean(fitness)> mean_best:
                        mean_best=np.mean(fitness)

                    # update 
                    convergence_curve['fitness'][iter_no] = mean_best
                    # display(harmonyMemory, fitness, agent_name)
                
                # compute final accuracy
                Leader_agent, Leader_accuracy = sort_agents(Leader_agent, compute_accuracy, data)
                harmonyMemory, accuracy = sort_agents(harmonyMemory, compute_accuracy, data)

                print('\n================================================================================')
                print('                                    Final Result                                  ')
                print('================================================================================\n')
                print('Leader ' + agent_name + ' Dimension : {}'.format(int(np.sum(Leader_agent))))
                print('Leader ' + agent_name + ' Fitness : {}'.format(Leader_fitness))
                print('Leader ' + agent_name + ' Classification Accuracy : {}'.format(Leader_accuracy))
                print('\n================================================================================\n')
                # print(Leader_agent)

                # leader agent and leader fitneess
                Leader_fitness = fitness[0]
                Leader_agent = harmonyMemory[0].copy()

                # stop timer
                end_time = time.time()
                exec_time = end_time - start_time

                # plot convergence graph
                fig, axes = Conv_plot(convergence_curve)
                if(save_conv_graph):
                    plt.savefig('convergence_graph_'+ short_name + '.jpg')
                plt.show()


                solution.best_agent = Leader_agent
                solution.best_fitness = Leader_fitness
                solution.best_accuracy = Leader_accuracy
                solution.convergence_curve = convergence_curve
                solution.final_population = harmonyMemory
                solution.final_fitness = fitness
                solution.final_accuracy = accuracy
                solution.execution_time = exec_time


    return solution



#UCI

dataset=''
df=pd.read_csv(dataset)
(a,b)=np.shape(df)
data = df.values[:,0:b-1]
label = df.values[:,b-1]

it=3
while it>0:
  predicted=HS(35, 60, data, label, save_conv_graph=True)
  it=it-1


